# Import Libraries

In [ ]:
install.packages("openxlsx")

library(readxl)
library(openxlsx)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
if (!requireNamespace("devtools", quietly = TRUE)) {
  install.packages("devtools")
}

devtools::install_github("fangzhou-xie/rethnicity")

library(rethnicity)

Rcpp       (NA -> 1.0.12   ) [CRAN]
RcppThread (NA -> 2.1.7    ) [CRAN]
RcppEigen  (NA -> 0.3.4.0.0) [CRAN]


Installing 3 packages: Rcpp, RcppThread, RcppEigen

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmppEilhu/remotescf2bfa0c97/fangzhou-xie-rethnicity-a513ca5/DESCRIPTION’ ... OK
* preparing ‘rethnicity’:
* checking DESCRIPTION meta-information ... OK
* cleaning src
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘rethnicity_0.2.4.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

══ WARNING! ════════════════════════════════════════════════════════════════════

The method provided by this package has its limitations and anyone must use them cautiously and responsibly. You should also agree that you only intend to use the method for academic research purpose and not for commercial use. You would also agree NOT to discriminate anyone based on race or color or any characteristic, with the information provided by this package. Please refer to the documentation for details: https://fangzhou-xie.github.io/rethnicity/index.html

════════════════════════════════════════════════════════════════════════════════



In [ ]:
data <- read.csv("data.csv")

# Detect Ethnicities

In [ ]:
predict_multiple_ethnicities <- function(names_lists) {
  results <- list()

  for (names in names_lists) {

    names <- na.omit(names)

    if (length(names) == 0) {
      results[[length(results) + 1]] <- character(0)
      next
    }


    split_names <- function(full_name) {
      name_parts <- unlist(strsplit(full_name, " "))
      if (length(name_parts) >= 2) {
        first_name <- name_parts[1]
        last_name <- paste(name_parts[-1], collapse = " ")
      } else {
        first_name <- full_name
        last_name <- ""
      }
      return(c(first_name, last_name))
    }


    names_split <- t(sapply(names, split_names))
    firstnames <- names_split[, 1]
    lastnames <- names_split[, 2]


    no_last_name_indices <- which(lastnames == "")
    if (length(no_last_name_indices) > 0) {
      lastnames[no_last_name_indices] <- "Unknown"
    }

    # Predict ethnicity using the rethnicity package
    predictions <- predict_ethnicity(firstnames, lastnames, method = "fullname")

    # Extract predicted ethnicities and combine into a single vector
    ethnicities_combined <- sapply(predictions$race, function(eth) paste(eth, collapse = ", "))

    results[[length(results) + 1]] <- ethnicities_combined
  }

  return(results)
}

## Get Authors' Ethnicity

In [ ]:
authorsNames <- strsplit(data[["Name"]], ", ")
authorsNames <- lapply(authorsNames, function(names) trimws(names))

In [ ]:
authorsEthnicities <- predict_multiple_ethnicities(authorsNames)
data$'Ethnicity' <- sapply(authorsEthnicities, function(eth) paste(eth, collapse = ", "))

## Get Google Scholar Co-Authors' Ethnicity

In [ ]:
scholarCoauthorsNames <- strsplit(data[["Co-authors’ names (Google Scholar)"]], ", ")
scholarCoauthorsNames <- lapply(scholarCoauthorsNames, function(names) trimws(names))

In [ ]:
scholarCoauthorsEthnicities <- predict_multiple_ethnicities(scholarCoauthorsNames)
data$'Co-authors’ ethnicity (Google Scholar)' <- sapply(scholarCoauthorsEthnicities, function(eth) paste(eth, collapse = ", "))

## Get OpenAI Co-Authors' Ethnicity

In [ ]:
openAIcoauthorsNames <- strsplit(data[["Co-authors’ names (OpenAI)"]], ", ")
openAIcoauthorsNames <- lapply(openAIcoauthorsNames, function(names) trimws(names))

In [ ]:
openAIcoauthorsEthnicities <- predict_multiple_ethnicities(openAIcoauthorsNames)
data$'Co-authors’ ethnicity (OpenAI)' <- sapply(openAIcoauthorsEthnicities, function(eth) paste(eth, collapse = ", "))

In [ ]:
write.xlsx(data, "ethnicityAddedData.xlsx", sheetName = "Sheet1", colNames = TRUE, rowNames = FALSE, append = TRUE)